In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
chromedriver ='/Users/youngseolee/Downloads/chromedriver'
driver = webdriver.Chrome(chromedriver)

In [5]:
# 임의의 주제로 검색한 후 View 클릭
base_url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=view&query='
url= base_url + quote('국내여행')

In [4]:
driver.get(url)

### 제목, 작성일자, 내용 데이터 프레임 구성, 파일로 저장

In [22]:
from selenium.webdriver.common.keys import Keys
body = driver.find_element_by_tag_name('body')
for i in range(20):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [9]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [23]:
lis = soup.select('li.bx._svp_item')
len(lis)

93

In [24]:
# 제목, 작성일자, 내용 데이터 
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit').get_text()
content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
date = li.select_one('.source_txt.date').get_text()

title, date, content

('제주투어패스 48시간 프리패스권',
 '2021.03.23.',
 '제주여행을 준비하는 사람이라면 설렘만큼 초조함도 크기 마련! 하지만 걱정 마시라. 여행 초보도 고수로 둔갑시켜줄 제주투어패스가 있으니 2박 3일동안 관광지 프리패스 가능! 플러스 제주핫플 카페까지!...')

In [25]:
title_list, content_list, date_list = [],[],[]
for li in lis:
    try:  
        title = li.select_one('a.api_txt_lines.total_tit').get_text()
        content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
        date = li.select_one('.source_txt.date').get_text()

    except:
        title_list.append(title)
        content_list.append(content)
        date_list.append(date)

In [28]:
df = pd.DataFrame({
    'title':title_list, 'date': date_list, 'content':content_list
})
df.head()

,title,date,content
0,국내여행 6월 거제 저구항 수국 등 거제 드라이브 코스,2021.03.16.,거제도 여행 중에 찾은 수국명소들. 6월 국내여행은 단연 수국꽃이 대표적인데요. 제...
1,국내 여행 추천 강원도 양구 가볼만한곳 BEST7,2021.03.16.,국내 여행 추천 명소 박수근 공원 안쪽으로 들어서면 박수근 라키비움 전시관으로 향할...
2,국내 여행지 추천 제주도 에코랜드 제주 테마파크,2021.03.16.,숲속 기차 여행을 할 수 있는 제주도 에코랜드는 기차를 좋아하는 아이들의 호기심을 ...
3,국내 여행지 추천 전남 가볼만한곳 여수에서 남중권시티투어,2021.03.16.,떠나는 국내 여행지 추천하게 만들었다. 총 9개 코스 중에서 남중권 누비 GO 4....
4,5월 국내 여행지 추천 아름다운 부산 볼거리,2021.03.16.,"오늘은 5월 국내 여행지 추천 장소로 부산 여행지를 정리해 보여드렸는데, 5월에 부..."


In [27]:
df.shape

(87, 3)

In [29]:
df.to_csv('travel.txt')

In [30]:
driver.close()